<a href="https://colab.research.google.com/github/aymuos/starship/blob/main/fundamental_EDA/actuallyWorking/feature_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install duckdb polars pyarrow

## Objective

- create features using both the datasets
-

In [3]:
base = "/content/drive/MyDrive/ml/PROCESSED/matched/city_divided/"

idea : trajectory[t−Δ , t]  →  delivery(t) where t = receipt_time
Δ = 15 minutes (initial)

In [4]:
import polars as pl

cq_delivery = "/content/drive/MyDrive/ml/PROCESSED/matched/city_divided/chongqing_data.csv"
cq_gps = "/content/drive/MyDrive/ml/PROCESSED/matched/city_divided/chongqing_delivery_data.parquet"

# Load the CSV file
chongqing_df_csv = pl.read_csv(cq_delivery)
print("\nChongqing CSV DataFrame (head 5):")
print(chongqing_df_csv.head(5))

# Load the Parquet file
chongqing_df_parquet = pl.read_parquet(cq_gps)
print("\nChongqing Parquet DataFrame (head 5):")
print(chongqing_df_parquet.head(5))


Chongqing CSV DataFrame (head 5):
shape: (5, 16)
┌─────────────┬─────────────┬────────────┬──────────┬───┬──────────┬─────┬────────────┬────────────┐
│ order_id    ┆ from_dipan_ ┆ delivery_u ┆ poi_lng  ┆ … ┆ sign_lat ┆ ds  ┆ from_city_ ┆ horizon_ET │
│ ---         ┆ id          ┆ ser_id     ┆ ---      ┆   ┆ ---      ┆ --- ┆ en         ┆ A          │
│ str         ┆ ---         ┆ ---        ┆ f64      ┆   ┆ str      ┆ i64 ┆ ---        ┆ ---        │
│             ┆ str         ┆ str        ┆          ┆   ┆          ┆     ┆ str        ┆ f64        │
╞═════════════╪═════════════╪════════════╪══════════╪═══╪══════════╪═════╪════════════╪════════════╡
│ 55be8cdf127 ┆ c5ac5ba9980 ┆ df0b594618 ┆ 8.8999e6 ┆ … ┆ null     ┆ 318 ┆ Chongqing  ┆ 361.0      │
│ 0526231c9ba ┆ 1aa6b85ba47 ┆ d1ba6f619e ┆          ┆   ┆          ┆     ┆            ┆            │
│ 3387f51b…   ┆ 3d926051…   ┆ 4e7dd03444 ┆          ┆   ┆          ┆     ┆            ┆            │
│             ┆             ┆ …          

In [5]:
chongqing_df_csv = chongqing_df_csv.with_columns([
    pl.col("receipt_time").str.to_datetime("%Y-%m-%d %H:%M:%S").alias("receipt_time"),
    pl.col("sign_time").str.to_datetime("%Y-%m-%d %H:%M:%S").alias("sign_time")
])

print("Chongqing CSV DataFrame with updated time column types:")
print(chongqing_df_csv.head(5))

Chongqing CSV DataFrame with updated time column types:
shape: (5, 16)
┌─────────────┬─────────────┬────────────┬──────────┬───┬──────────┬─────┬────────────┬────────────┐
│ order_id    ┆ from_dipan_ ┆ delivery_u ┆ poi_lng  ┆ … ┆ sign_lat ┆ ds  ┆ from_city_ ┆ horizon_ET │
│ ---         ┆ id          ┆ ser_id     ┆ ---      ┆   ┆ ---      ┆ --- ┆ en         ┆ A          │
│ str         ┆ ---         ┆ ---        ┆ f64      ┆   ┆ str      ┆ i64 ┆ ---        ┆ ---        │
│             ┆ str         ┆ str        ┆          ┆   ┆          ┆     ┆ str        ┆ f64        │
╞═════════════╪═════════════╪════════════╪══════════╪═══╪══════════╪═════╪════════════╪════════════╡
│ 55be8cdf127 ┆ c5ac5ba9980 ┆ df0b594618 ┆ 8.8999e6 ┆ … ┆ null     ┆ 318 ┆ Chongqing  ┆ 361.0      │
│ 0526231c9ba ┆ 1aa6b85ba47 ┆ d1ba6f619e ┆          ┆   ┆          ┆     ┆            ┆            │
│ 3387f51b…   ┆ 3d926051…   ┆ 4e7dd03444 ┆          ┆   ┆          ┆     ┆            ┆            │
│             ┆     

In [6]:
print("Chongqing CSV DataFrame schema after datetime conversion:")
print(chongqing_df_csv.schema)

Chongqing CSV DataFrame schema after datetime conversion:
Schema({'order_id': String, 'from_dipan_id': String, 'delivery_user_id': String, 'poi_lng': Float64, 'poi_lat': Float64, 'aoi_id': String, 'typecode': String, 'receipt_time': Datetime(time_unit='us', time_zone=None), 'receipt_lng': Float64, 'receipt_lat': Float64, 'sign_time': Datetime(time_unit='us', time_zone=None), 'sign_lng': String, 'sign_lat': String, 'ds': Int64, 'from_city_en': String, 'horizon_ETA': Float64})


In [7]:
chongqing_df_csv = chongqing_df_csv.drop(["sign_lat", "sign_lng"])

print("Chongqing CSV DataFrame after dropping sign_lat and sign_lng (head 5):")
print(chongqing_df_csv.head(5))

print("\nChongqing CSV DataFrame schema after dropping sign_lat and sign_lng:")
print(chongqing_df_csv.schema)

Chongqing CSV DataFrame after dropping sign_lat and sign_lng (head 5):
shape: (5, 14)
┌────────────┬────────────┬────────────┬──────────┬───┬────────────┬─────┬────────────┬────────────┐
│ order_id   ┆ from_dipan ┆ delivery_u ┆ poi_lng  ┆ … ┆ sign_time  ┆ ds  ┆ from_city_ ┆ horizon_ET │
│ ---        ┆ _id        ┆ ser_id     ┆ ---      ┆   ┆ ---        ┆ --- ┆ en         ┆ A          │
│ str        ┆ ---        ┆ ---        ┆ f64      ┆   ┆ datetime[μ ┆ i64 ┆ ---        ┆ ---        │
│            ┆ str        ┆ str        ┆          ┆   ┆ s]         ┆     ┆ str        ┆ f64        │
╞════════════╪════════════╪════════════╪══════════╪═══╪════════════╪═════╪════════════╪════════════╡
│ 55be8cdf12 ┆ c5ac5ba998 ┆ df0b594618 ┆ 8.8999e6 ┆ … ┆ 2021-03-18 ┆ 318 ┆ Chongqing  ┆ 361.0      │
│ 70526231c9 ┆ 01aa6b85ba ┆ d1ba6f619e ┆          ┆   ┆ 14:33:00   ┆     ┆            ┆            │
│ ba3387f51b ┆ 473d926051 ┆ 4e7dd03444 ┆          ┆   ┆            ┆     ┆            ┆            │
│ …  

Feature creation starts
---


---


In [8]:
delivery = (
    chongqing_df_csv
    .rename({"horizon_ETA": "eta_mins"})
    .with_columns(
        pl.col("delivery_user_id").cast(pl.Utf8)
    )
    .sort(["delivery_user_id", "receipt_time"])
)

In [9]:
# defining just the gps extraction windows

start_time = delivery["receipt_time"].min()
end_time   = delivery["receipt_time"].max()

couriers = delivery["delivery_user_id"].unique().to_list()

print(len(couriers), "couriers")

145 couriers


In [10]:
import duckdb

con = duckdb.connect()

PRE_MIN = 15

gps_df = con.execute(f"""
SELECT
    postman_id,
    gps_time,
    lat,
    lng
FROM parquet_scan('/content/drive/MyDrive/ml/PROCESSED/matched/city_divided/chongqing_delivery_data.parquet')
WHERE
    postman_id IN {tuple(couriers)}
    AND gps_time BETWEEN
        TIMESTAMP '{start_time}' - INTERVAL '{PRE_MIN} minutes'
        AND TIMESTAMP '{end_time}'
""").pl()

gps_df = gps_df.rename({"postman_id": "delivery_user_id"})
gps_df = gps_df.sort(["delivery_user_id", "gps_time"])

In [11]:
# Courier Snapshot Before Delivery (ASOF JOIN)

state = delivery.join_asof(
    gps_df,
    left_on="receipt_time",
    right_on="gps_time",
    by="delivery_user_id",
    strategy="backward"
).rename({
    "lat": "last_x",
    "lng": "last_y",
    "gps_time": "last_gps_time"
})

/tmp/ipython-input-950023143.py:3: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided
  state = delivery.join_asof(


In [12]:
# GPS Freshness Feature
state = state.with_columns(
    (pl.col("receipt_time") - pl.col("last_gps_time"))
        .dt.total_minutes()
        .alias("gps_gap_min")
)

In [13]:
state.select("gps_gap_min").describe()

statistic,gps_gap_min
str,f64
"""count""",19062.0
"""null_count""",7608.0
"""mean""",315.771168
"""std""",1520.738809
"""min""",0.0
"""25%""",0.0
"""50%""",0.0
"""75%""",0.0
"""max""",15808.0


gps_time ≤ receipt_time is ensured by asof_join . Invalidate stale gps matches : GPS older than 30 minutes → unusable

-

In [14]:
MAX_GAP = 30  # minutes

state = state.with_columns(
    pl.when(pl.col("gps_gap_min") <= MAX_GAP)
      .then(pl.col("gps_gap_min"))
      .otherwise(None)
      .alias("gps_gap_min")
)

In [15]:
state = state.with_columns([
    pl.when(pl.col("gps_gap_min").is_null())
      .then(None)
      .otherwise(pl.col("last_x"))
      .alias("last_x"),

    pl.when(pl.col("gps_gap_min").is_null())
      .then(None)
      .otherwise(pl.col("last_y"))
      .alias("last_y"),
])


# idea is to prevent stale gps from corrupting matches

In lade Delivery coverage > GPS coverage , So GPS must be treated as:
optional contextual sensor not guaranteed observation. This will influence later bandit formulation.

In [16]:
# diagonistic code

state.select([
    pl.len().alias("rows"),
    pl.col("gps_gap_min").null_count().alias("missing_gps"),
    pl.mean("gps_gap_min").alias("mean_gap"),
    pl.median("gps_gap_min").alias("median_gap"),
])

rows,missing_gps,mean_gap,median_gap
u32,u32,f64,f64
26670,9472,0.324689,0.0


This means :

- rows          = 26670 deliveries
- missing_gps   = 9472  (~35%)
- mean_gap      = 0.32 minutes
- median_gap    = 0.0 minutes

---

- Most deliveries have a GPS sample almost exactly at receipt_time.
- mean_gap < 1 minute i.e stale gps are removed


# Building trajectory window

In [29]:
# Registering table in duckDB

con.register("delivery_tbl", delivery.to_pandas())

In [30]:
PRE_MIN = 15

traj_features = con.execute(f"""
WITH gps_window AS (

    SELECT
        d.order_id,
        g.gps_time,
        g.lat,
        g.lng,

        LAG(g.lat) OVER (
            PARTITION BY d.order_id
            ORDER BY g.gps_time
        ) AS prev_lat,

        LAG(g.lng) OVER (
            PARTITION BY d.order_id
            ORDER BY g.gps_time
        ) AS prev_lng,

        LAG(g.gps_time) OVER (
            PARTITION BY d.order_id
            ORDER BY g.gps_time
        ) AS prev_time

    FROM delivery_tbl d
    JOIN parquet_scan('{cq_gps}') g
      ON g.postman_id = d.delivery_user_id

    WHERE g.gps_time BETWEEN
        d.receipt_time - INTERVAL '{PRE_MIN} minutes'
        AND d.receipt_time
),

motion AS (

    SELECT
        order_id,

        SQRT(
            POWER(lat - prev_lat, 2) +
            POWER(lng - prev_lng, 2)
        ) AS dist,

        EXTRACT(EPOCH FROM (gps_time - prev_time)) AS dt

    FROM gps_window
),

speed_calc AS (


    SELECT
        order_id,
        dist,
        dt,

        CASE
            WHEN dt IS NULL THEN NULL
            WHEN dt <= 1 THEN NULL        -- remove tiny intervals
            WHEN dist > 5000 THEN NULL   -- teleportation filter
            WHEN dist / dt > 5 THEN NULL   -- speed cap for not natural speed
            ELSE dist / dt
        END AS speed

    FROM motion
)

SELECT
    order_id,
    COUNT(*) AS gps_points,
    AVG(speed) AS speed_mean,
    SUM(dist) AS distance_travelled

FROM speed_calc
GROUP BY order_id
""").pl()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [31]:
traj_features.head()

order_id,gps_points,speed_mean,distance_travelled
str,i64,f64,f64
"""032b481069ffcf68c3e496e1e1a109…",45,0.476512,419.330758
"""05ecbccff4b356621dafb950524f6e…",45,0.18789,165.342807
"""15638e2617916788e057cec3f8f5b6…",45,0.664417,584.686812
"""2180eafca8ca86da7a46308f3faeb1…",16,0.243197,175.938474
"""236407035e531bf32bcbd450668677…",45,0.502854,442.511723


dt <= 1

GPS sampling ≈ 20 seconds.

Anything ≤1 sec is noise.

dist > 5000

Affine coordinates behave like meters-scale space. A courier cannot move 5 km between samples.

This removes privacy jumps.

In [32]:
traj_features.select([
    pl.max("speed_mean").alias("max_speed_mean"),
    pl.mean("speed_mean").alias("mean_speed_mean")
])

max_speed_mean,mean_speed_mean
f64,f64
2.967034,0.492363


conclusion from this is :
missing GPS samples
→ courier reappears far away
→ large dist but moderate dt


Instead of trying to perfectly clean GPS (impossible), we winsorize speeds.

This is standard in:

Uber ETA systems

mobility forecasting

trajectory ML papers

This adding of limit of 5 helped us do trajectory signal extraction
NOT GPS reconstruction

In [33]:
features = (
    delivery
      .join(state.select(["order_id","last_x","last_y","gps_gap_min"]), on="order_id", how="left")
      .join(traj_features, on="order_id", how="left")
)

merging features

In [35]:
# adding temporal context

features = features.with_columns([
    pl.col("receipt_time").dt.hour().alias("hour"),
    pl.col("receipt_time").dt.weekday().alias("weekday"),

    pl.col("order_id").cum_count().over("delivery_user_id")
        .alias("delivery_sequence")
])

In [36]:
features.schema

Schema([('order_id', String),
        ('from_dipan_id', String),
        ('delivery_user_id', String),
        ('poi_lng', Float64),
        ('poi_lat', Float64),
        ('aoi_id', String),
        ('typecode', String),
        ('receipt_time', Datetime(time_unit='us', time_zone=None)),
        ('receipt_lng', Float64),
        ('receipt_lat', Float64),
        ('sign_time', Datetime(time_unit='us', time_zone=None)),
        ('ds', Int64),
        ('from_city_en', String),
        ('eta_seconds', Float64),
        ('last_x', Float64),
        ('last_y', Float64),
        ('gps_gap_min', Int64),
        ('gps_points', Int64),
        ('speed_mean', Float64),
        ('distance_travelled', Float64),
        ('hour', Int8),
        ('weekday', Int8),
        ('delivery_sequence', UInt32)])

In [37]:
features.head()

order_id,from_dipan_id,delivery_user_id,poi_lng,poi_lat,aoi_id,typecode,receipt_time,receipt_lng,receipt_lat,sign_time,ds,from_city_en,eta_seconds,last_x,last_y,gps_gap_min,gps_points,speed_mean,distance_travelled,hour,weekday,delivery_sequence
str,str,str,f64,f64,str,str,datetime[μs],f64,f64,datetime[μs],i64,str,f64,f64,f64,i64,i64,f64,f64,i8,i8,u32
"""7b15e523a08ad352b9f22bb4ecd998…","""37e976ad4abb10da92c97c6b34f7f5…","""0008c2b6a2314db8715301b7eeeebc…",8.9031e6,-7.6873e6,"""bfac06c1b710a20d8dcfb8d3093d7e…","""203ac3454d75e02ebb0a3c6f51d735…",2021-03-18 08:20:00,8.9041e6,-7.6866e6,2021-03-18 11:51:00,318,"""Chongqing""",211.0,null,null,null,null,null,null,8,4,1
"""3d7ca50514737df23244a2c633b7ee…","""37e976ad4abb10da92c97c6b34f7f5…","""0008c2b6a2314db8715301b7eeeebc…",8.9035e6,-7.6870e6,"""8ceca456f58ab1d2595d571a2205c5…","""73ffcbd1b26557b462b14e4dd4c57f…",2021-03-18 08:21:00,8.9041e6,-7.6866e6,2021-03-18 11:26:00,318,"""Chongqing""",185.0,null,null,null,null,null,null,8,4,2
"""5134623a6ae48bf37cc58bbcb22412…","""37e976ad4abb10da92c97c6b34f7f5…","""0008c2b6a2314db8715301b7eeeebc…",8.9028e6,-7.6875e6,"""8b8a66982b68f85ed1d8e56dfe3144…","""73ffcbd1b26557b462b14e4dd4c57f…",2021-03-18 08:21:00,8.9041e6,-7.6866e6,2021-03-18 18:19:00,318,"""Chongqing""",598.0,null,null,null,null,null,null,8,4,3
"""95334065d9951e356138196715ba21…","""37e976ad4abb10da92c97c6b34f7f5…","""0008c2b6a2314db8715301b7eeeebc…",8.9032e6,-7.6870e6,"""35ba604f0afd85d280cb9d96a69874…","""203ac3454d75e02ebb0a3c6f51d735…",2021-03-18 08:21:00,8.9041e6,-7.6866e6,2021-03-18 18:33:00,318,"""Chongqing""",612.0,null,null,null,null,null,null,8,4,4
"""4767aafe217550daac5895cbc46b46…","""37e976ad4abb10da92c97c6b34f7f5…","""0008c2b6a2314db8715301b7eeeebc…",8.9030e6,-7.6871e6,"""594c43284e8d7dedd4fcf06106379f…","""203ac3454d75e02ebb0a3c6f51d735…",2021-03-18 08:23:00,8.9041e6,-7.6866e6,2021-03-18 12:18:00,318,"""Chongqing""",235.0,null,null,null,null,null,null,8,4,5


In [38]:
features.select([
    pl.len().alias("rows"),
    pl.mean("gps_points"),
    pl.mean("speed_mean"),
    pl.mean("eta_seconds")
])

rows,gps_points,speed_mean,eta_seconds
u32,f64,f64,f64
26670,34.390476,0.492363,285.018823
